In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2021-12-01 12:28:41--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.84.168.111, 2600:1407:3c00:10a2::e59, 2600:1407:3c00:108c::e59
Connecting to download.microsoft.com (download.microsoft.com)|184.84.168.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   122MB/s    in 6.9s    

2021-12-01 12:28:48 (115 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [ ]:
!ls

drive  kagglecatsanddogs_3367a.zip  sample_data


In [ ]:
!unzip kagglecatsanddogs_3367a.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: PetImages/Dog/5500.jpg  
  inflating: PetImages/Dog/5501.jpg  
  inflating: PetImages/Dog/5502.jpg  
  inflating: PetImages/Dog/5503.jpg  
  inflating: PetImages/Dog/5504.jpg  
  inflating: PetImages/Dog/5505.jpg  
  inflating: PetImages/Dog/5506.jpg  
  inflating: PetImages/Dog/5507.jpg  
  inflating: PetImages/Dog/5508.jpg  
  inflating: PetImages/Dog/5509.jpg  
  inflating: PetImages/Dog/551.jpg   
  inflating: PetImages/Dog/5510.jpg  
  inflating: PetImages/Dog/5511.jpg  
  inflating: PetImages/Dog/5512.jpg  
  inflating: PetImages/Dog/5513.jpg  
  inflating: PetImages/Dog/5514.jpg  
  inflating: PetImages/Dog/5515.jpg  
  inflating: PetImages/Dog/5516.jpg  
  inflating: PetImages/Dog/5517.jpg  
  inflating: PetImages/Dog/5518.jpg  
  inflating: PetImages/Dog/5519.jpg  
  inflating: PetImages/Dog/552.jpg   
  inflating: PetImages/Dog/5520.jpg  
  inflating: PetImages/Dog/5521.jpg  
  inflating: PetImag

In [ ]:
!ls /content/PetImages/Cat

In [ ]:
!find /content/PetImages/Cat

In [ ]:
!find /content/PetImages/Cat

In [ ]:
!ls /content/PetImages/Cat -1 | wc -l

12501


In [ ]:
import shutil
import os

In [ ]:
data_dir = '/content/PetImages/'
train_dir = 'train'
val_dir   = 'val'
test_dir  = 'test'
test_portion = 0.15
val_portion = 0.15
nb_images = 12500

In [ ]:
def create_dir(dir_name):
  
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)

  os.makedirs(dir_name)
  os.makedirs(os.path.join(dir_name, "cats"))
  os.makedirs(os.path.join(dir_name, "dogs"))

In [ ]:
create_dir(train_dir)
create_dir(test_dir)
create_dir(val_dir)

In [ ]:
def copy_images(st, ed, src, dst):
  for i in range(st, ed):
    shutil.copy2(os.path.join(src, "Cat/" + str(i) + ".jpg"), os.path.join(dst, "cats"))
    shutil.copy2(os.path.join(src, "Dog/" + str(i) + ".jpg"), os.path.join(dst, "dogs"))

In [ ]:
start_val = int(nb_images * (1 - val_portion - test_portion))
start_tst = int(nb_images * (1 - test_portion))
print(start_val)
print(start_tst)
# train - 0..8749
# val   - 8750 .. 10624
# test  - 10625 .. 12500

8750
10625


In [ ]:
# train
copy_images(0, start_val, data_dir, train_dir)

In [ ]:
copy_images(start_val, start_tst, data_dir, val_dir)

In [ ]:
copy_images(start_tst, nb_images, data_dir, test_dir)

In [ ]:
#%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Activation, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 2
batch_size = 128
nb_train = 17500
nb_val   = 3750
nb_test  = 3750

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = datagen.flow_from_directory(train_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 17500 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(val_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 3750 images belonging to 2 classes.


In [ ]:
test_generator = datagen.flow_from_directory(test_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 3750 images belonging to 2 classes.


In [ ]:
import glob
from PIL import Image

images = glob.glob("/content/train/cats/*.jpg")

for image in images:
    with open(image, 'rb') as file:
        im = Image.open(file)
        print(image, im.width, im.height, im.mode, im.format, type(im))

Выходные данные были обрезаны до нескольких последних строк (5000).
/content/train/cats/5977.jpg 500 375 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/2651.jpg 500 356 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/6325.jpg 500 414 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/4327.jpg 307 409 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/4383.jpg 420 360 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/8077.jpg 500 466 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/7736.jpg 476 500 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/4543.jpg 357 500 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/7867.jpg 329 335 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/180.jpg 349 500 RGB JPEG <class 'PIL.JpegImagePlugin.JpegImageFile'>
/content/train/cats/3.jpg 500 415

In [ ]:
!cp /content/train/cats/667.jpg /content/train/cats/666.jpg

In [ ]:
!cp /content/test/dogs/11703.jpg /content/test/dogs/11702.jpg

In [ ]:
# https://stackoverflow.com/questions/60168905/pil-unidentifiedimageerror-cannot-identify-image-file
model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train // batch_size,
                    epochs = epochs,
                    validation_data  = val_generator,
                    validation_steps = nb_val // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/2
136/136 [==============================] - ETA: 0s - loss: 0.6476 - accuracy: 0.6105

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

136/136 [==============================] - 71s 524ms/step - loss: 0.6476 - accuracy: 0.6105 - val_loss: 0.6160 - val_accuracy: 0.6853
Epoch 2/2
136/136 [==============================] - 70s 513ms/step - loss: 0.5760 - accuracy: 0.7008 - val_loss: 0.5356 - val_accuracy: 0.7266


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
scores

[0.544794499874115, 0.7163254022598267]